In [80]:
import pandas as pd
import geopandas as gpd
import folium
import folium.plugins as plugins
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import interact

In [81]:
weather_cities = pd.read_csv('results/cities.csv')
hotels = pd.read_json('results/top20-hotels.json')
hotels = pd.merge(hotels, weather_cities, how='inner', left_on='city_id', right_on='city_id', suffixes=('','_city'))
#weather_cities = gpd.GeoDataFrame(weather_cities, geometry = gpd.points_from_xy(weather_cities.lon, weather_cities.lat), crs='EPSG:4326')

# Map of the weather forecast

In [82]:
def temp_category(temp):
    if temp <= 14 :
        return '<= 14°C'
    elif temp >= 18 :
        return '>= 18°C'
    else :
        return '[15°C , 17°C]'

In [83]:
weather_cities['temp_category']= weather_cities['mean_feels_like'].apply(lambda x : temp_category(x))

In [84]:
m = folium.Map(location=[weather_cities['lat'].mean(), weather_cities['lon'].mean()], tiles='CartoDB positron', zoom_start=5, width=500,height=500)

#geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in weather_cities.geometry]


for i in range(0,len(weather_cities)) :
    # assign a color marker
    if weather_cities.iloc[i]['temp_category'] == '<= 14°C':
        type_color = 'lightgreen'
        icon_name = 'umbrella'
    elif weather_cities.iloc[i]['temp_category'] == '[15°C , 17°C]':
        type_color = 'beige'
        icon_name = 'sun-o'
    else:
        type_color = 'orange'
        icon_name = 'sun-o'
    # assing marker icon
    if weather_cities.iloc[i]['mean_pop'] == 0:
        icon_name = 'sun-o'
    else:
        icon_name = 'umbrella'

    # Place the markers with the popup labels and data
    m.add_child(
        folium.Marker(
            location=[weather_cities.iloc[i]['lat'], weather_cities.iloc[i]['lon']],
            popup= folium.Popup(
                "City: " + str(weather_cities.iloc[i]['city']) + "<br>"
                + "Mean temperature in the next 7 days: " + str(weather_cities.iloc[i]['mean_feels_like']) + "<br>"
                + "Probability of rain in the next 7 days: " + str(weather_cities.iloc[i]['mean_pop']) ,
                max_width=300,
                min_width=300  
            ),
            icon=folium.Icon(color=type_color, icon=icon_name, prefix='fa', opacity=0.5),
        )
    )
    
title_html = '''
             <h3 align="left" style="font-size:15px"><b>Weather forecast for the next 7 days</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))
m.save('results/weather_forecast.html')
m

# Top 5 destination

In [85]:
top5 = weather_cities.loc[:4]
top5

,city_id,city,lat,lon,mean_feels_like,mean_pop,temp_category
0,0,Cassis,43.214036,5.539632,19,0.0,>= 18°C
1,1,Collioure,42.525050,3.083155,19,0.0,>= 18°C
2,2,Carcassonne,43.213036,2.349107,19,0.0,>= 18°C
3,3,Marseille,43.296174,5.369953,18,0.0,>= 18°C
4,4,Aix en Provence,43.529842,5.447474,18,0.0,>= 18°C


In [86]:
m = folium.Map(location=[top5['lat'].mean(), top5['lon'].mean()], tiles='CartoDB positron', zoom_start=7 , width=500,height=500)
    
for i in range(0,len(top5)):
    
    folium.Marker(
        location=[top5.iloc[i]['lat'], top5.iloc[i]['lon']], 
        popup=top5.iloc[i]['city'],
        icon=plugins.BeautifyIcon(
                        icon_shape="marker",
                         number= i + 1,
                         border_color = '#FFC300',
                         background_color = '#FFC300'
                     )
    ).add_to(m)
    
title_html = '''
             <h3 align="left" style="font-size:15px"><b>Top 5 destinations for the next 7 days</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))
m.save('results/top5_destinations.html')
# Show the map again
m

# Top 20 hotels in the top 5 cities

In [87]:
def show_map(city):
    hotels_selection = hotels[hotels['city'] == city]
    
    m = folium.Map(location=[hotels_selection['lat'].mean(), hotels_selection['lon'].mean()], tiles='CartoDB positron', zoom_start=14, width=900,height=500)
    marker_cluster = plugins.MarkerCluster(name="Top 20 hotels").add_to(m)
    
    for i in range(0,len(hotels_selection)):
        folium.Marker(
            location=[hotels_selection.iloc[i]['lat'], hotels_selection.iloc[i]['lon']], 
            popup= folium.Popup(
                "Name: " + str(hotels_selection.iloc[i]['name']) + "<br>"
                + "Address: " + str(hotels_selection.iloc[i]['address']) + "<br>"
                + "Rating: " + str(hotels_selection.iloc[i]['rating']) + "<br>"
                + "Score: " + str(hotels_selection.iloc[i]['score']) ,
                max_width=300,
                min_width=300  
            ),
            icon=folium.Icon(color="red", icon="hotel", prefix='fa')
        ).add_to(marker_cluster)
    
    marker_cluster.add_to(m)
    
    display(m)

In [88]:
city_w = widgets.Dropdown(options=top5['city'].values, description="Choose a city")
interact(show_map, city = city_w)

interactive(children=(Dropdown(description='Choose a city', options=('Cassis', 'Collioure', 'Carcassonne', 'Ma…

<function __main__.show_map(city)>